In [1]:
import sys
sys.path.append('../scripts')

import chromedriver_autoinstaller

import shutil

import os
from utils import cleanCmpnyNm, getGrpCmpnyDict, convertCmpnyNm, convertCmpnyNm
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
import re

In [2]:
chromedriver_autoinstaller.install(path="../utils/")

'/Users/choidamian/Chaebol/utils/102/chromedriver'

In [3]:
inputData = []
for file in os.listdir('../data/ownership-input/2021/'):
    if not file.endswith('raw.xlsx'):
        continue
    df = pd.read_excel('../data/ownership-input/2021/'+file)
    inputData.append(df)
dfOwnInputMrg = pd.concat(inputData)
dfOwnInputMrg.head()

,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode
0,그로쓰파트너,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,금융회사,미래에셋,miraeasset,2602105
1,멀티에셋자산운용,0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105
2,멀티에셋자산운용,1,1.000000,1.000000,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105
3,멀티에셋자산운용,99,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105
4,미래에셋글로벌유니콘사모투자,1,0.867857,0.867857,미래에셋증권,금융회사,미래에셋,miraeasset,2602105


In [4]:
dfOwnInputMrg['grpname'].nunique()

60

In [5]:
def convertCmpnyNm(row):
    
    out = row['firmname']
    
    if row['grpname2'] == '엘에스':
        out = re.sub("LS", "엘에스", out)
        out = re.sub("E1", "이원", out)
        out = re.sub("KJ", "케이제이", out)
        out = re.sub("JS", "제이에스", out)
        
    elif row['grpname2'] == '미래에셋':
        out = re.sub('미래에셋파트너스제9호사모투자', '미래에셋파트너스제구호사모투자', out)
        out = re.sub('미래에셋파트너스9호', '미래에셋파트너스제구호사모투자', out)
        
    elif row['grpname2'] == '태영':
        out = re.sub('SBS', '에스비에스', out)
        out = re.sub('KCP', '케이씨피', out)
        out = re.sub('PFV', '피에프브이', out)
        
    elif row['grpname2'] == 'DB':
        out = re.sub('Inc\.*$', '아이엔씨', out)
        out = re.sub('DB', '디비', out)
        
    elif row['grpname2'] == 'KG':
        out = re.sub('KG', '케이지', out)
        
    elif row['grpname2'] == '하림':
        out = re.sub('FS', '에프에스', out)
    
    return out

In [6]:
dfOwnInputMrg['firmnameConv'] = dfOwnInputMrg.apply(convertCmpnyNm, axis=1)
dfOwnInputMrg['firmnameConv'] = dfOwnInputMrg['firmnameConv'].replace("^자중흥주택$", '중흥주택', regex=True)
dfOwnInputMrg['firmnameConv'] = dfOwnInputMrg['firmnameConv'].replace("^비에이엠시스템폐업$", '비에이엠시스템', regex=True)
dfOwnInputMrg['firmnameConv'] = dfOwnInputMrg['firmnameConv'].replace("구,이사야에이치피피", '', regex=True)
dfOwnInputMrg['firmnameConv'] = dfOwnInputMrg['firmnameConv'].replace("\.에스케이인포섹", '', regex=True)
dfOwnInputMrg['firmnameConv'] = dfOwnInputMrg['firmnameConv'].replace("에이케이에셋$", '', regex=True)
dfOwnInputMrg

,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode,firmnameConv
0,그로쓰파트너,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,금융회사,미래에셋,miraeasset,2602105,그로쓰파트너
1,멀티에셋자산운용,0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용
2,멀티에셋자산운용,1,1.000000,1.000000,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용
3,멀티에셋자산운용,99,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용
4,미래에셋글로벌유니콘사모투자,1,0.867857,0.867857,미래에셋증권,금융회사,미래에셋,miraeasset,2602105,미래에셋글로벌유니콘사모투자
...,...,...,...,...,...,...,...,...,...,...
108,코스파,1,0.100000,0.100000,에이케이아이에스,비금융회사,애경,AK,615,코스파
109,코스파,99,0.500000,0.500000,NaN,비금융회사,애경,AK,615,코스파
110,퍼시픽제3호전문사모부동산투자,0,0.000000,0.000000,NaN,비금융회사,애경,AK,615,퍼시픽제3호전문사모부동산투자
111,퍼시픽제3호전문사모부동산투자,1,0.998586,0.998586,제주항공,비금융회사,애경,AK,615,퍼시픽제3호전문사모부동산투자


In [7]:
year = 2021
dfCmpnySumry = pd.read_excel(f'../data/cmpnySumry/cmpnySumry{year}Kor.xlsx', thousands=',')
dfCmpnySumry.head()

,공개년월,회계년,기업집단명,소속회사명,대표자,설립일,계열편입일,업종코드,영위업종,종업원수,...,자산총액,부채총액,자본총액,자본금,매출액,당기순이익,구분,법인등록번호,year2,rcmgCode
0,202105,2020,롯데,롯데쇼핑,강희태,19700702,19870401.0,G47,소매업; 자동차 제외,24963,...,2.490000e+13,1.530000e+13,9.600000e+12,1.410000e+11,8.710000e+12,-1.030000e+12,일반회사,1100000000000,2021,620068
1,202105,2020,지에스,지에스건설,임병용,19691219,20050404.0,F41,종합 건설업,6928,...,1.190000e+13,7.450000e+12,4.420000e+12,4.040000e+11,8.890000e+12,3.270000e+11,일반회사,1100000000000,2021,500143
2,202105,2020,SM,삼라,조유선,19690826,20040501.0,F41,종합 건설업,12,...,6.240000e+11,2.720000e+11,3.520000e+11,1.685000e+10,1.490000e+11,7.785100e+10,일반회사,1100000000000,2021,500429
3,202105,2020,한화,한화,옥경석 외,19521028,19870401.0,C20,화학 물질 및 화학제품 제조업; 의약품 제외,5026,...,7.360000e+12,3.870000e+12,3.490000e+12,4.900000e+11,4.000000e+12,1.290000e+11,일반회사,1100000000000,2021,350893
4,202105,2020,동원,동원산업,이명우,19690416,20030128.0,A03,어업,761,...,1.630000e+12,8.800000e+11,7.540000e+11,1.838800e+10,7.020000e+11,7.895000e+10,일반회사,1100000000000,2021,100030


In [8]:
dfCmpnySumry['firmnameConv'] = cleanCmpnyNm(dfCmpnySumry['소속회사명']).replace('\s', '', regex=True)

# dfCmpnySumry['firmnameConv'] = dfCmpnySumry['firmnameConv'].replace('합자회사', '', regex=True)
dfCmpnySumry['firmid'] = dfCmpnySumry['rcmgCode']
dfCmpnySumry['ownerid'] = dfCmpnySumry['rcmgCode']

In [9]:
dfOwnInputMrgFirmId = dfOwnInputMrg.merge(dfCmpnySumry[['firmnameConv', 'firmid']].drop_duplicates(), on='firmnameConv', how='left')
dfOwnInputMrgFirmId.head()

,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode,firmnameConv,firmid
0,그로쓰파트너,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,금융회사,미래에셋,miraeasset,2602105,그로쓰파트너,1.701406e+09
1,멀티에셋자산운용,0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05
2,멀티에셋자산운용,1,1.000000,1.000000,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05
3,멀티에셋자산운용,99,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05
4,미래에셋글로벌유니콘사모투자,1,0.867857,0.867857,미래에셋증권,금융회사,미래에셋,miraeasset,2602105,미래에셋글로벌유니콘사모투자,2.101110e+09


In [10]:
list(dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmid'].isna()]['firmnameConv'].unique())

['미래에셋큐리어스구조혁신기업재무안정사모투자',
 '미래에셋파트너스제구호사모투자',
 '동부철구',
 '유진에버베스트턴어라운드기업재무안정사모투자',
 '지구레미콘',
 '신세계동대구복합환승센터',
 '신세계야구단',
 '광주일보사',
 '탈리다쿰',
 '부신신항다목적터미널',
 '두산경영연구원',
 '삼성경제연구소',
 '수원삼성축구단',
 '대구남부에이엠씨',
 '천안풍세일반산업단지',
 '에이치지이니셔티브시몬느임팩트제1호창업베처전문사모투자',
 '아이디퀀티크유',
 '에스케이에코플랜트',
 '한국넥슬렌유',
 '트리엔파트너스',
 '중흥카이트제십구호위탁관리부동산투자회사',
 '롯데미래전략연구소',
 '롯데쇼핑타운대구',
 '마곡지구피에프브이',
 '유한책임회사성암허심',
 '자양5구역피에프브이',
 '프레시원대구경북',
 '아센디오',
 '이리츠코크렙기업구조조정부동산투자회사',
 '대한해운엘엔지',
 '에스에이엠씨투자대부',
 '골프와친구',
 '파괴연구소',
 '애경중부컨트리클럽']

In [11]:
dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '미래에셋큐리어스구조혁신기업재무안정사모투자'].index

Int64Index([47, 48, 49], dtype='int64')

In [12]:
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '미래에셋큐리어스구조혁신기업재무안정사모투자'].index, 'firmid']

47   NaN
48   NaN
49   NaN
Name: firmid, dtype: float64

In [13]:
dfCharToInt = pd.read_excel('../data/utils/char-to-int.xlsx')

In [14]:
username = 'wonbok'
password = 'gkrtkrhk5034!'

In [15]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

grsrch_url = 'https://www.kisline.com/cm/CM0100M00GE00.nice'
driver.get(grsrch_url)
driver.find_element(By.CLASS_NAME, 'btn_close_layer').click()
driver.find_element(By.ID, 'lgnuid').send_keys(username)
driver.find_element(By.ID, 'tmp_lgnupassword').click()
driver.find_element(By.ID, 'lgnupassword').send_keys(password)
driver.find_element(By.CLASS_NAME, 'btn_log_in').click()
driver.implicitly_wait(3)
sleep(1)

inputElem = driver.find_element_by_id('q')

for cmpny in list(dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmid'].isna()]['firmnameConv'].unique()):
    
    inputElem.send_keys(cmpny)
    driver.find_element(By.ID, 'searchView').click()
    driver.implicitly_wait(3)
    try:
        targetRow = driver.find_element(By.XPATH, '//*[@id="eprTable"]/tbody/tr/td[2]')
        kiscode = targetRow.get_attribute('data-kiscode')
        for i in range(len(dfCharToInt)):
            kiscode = re.sub(fr'{dfCharToInt.iloc[i, 0]}', str(dfCharToInt.iloc[i, 1]), kiscode)
        dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == cmpny].index, 'firmid'] = kiscode
        print(f"Found kiscode for {cmpny}: {kiscode}")
    except:
        print(f"No company found for {cmpny}. Continuing...")
        
    sleep(3)
    inputElem = driver.find_element(By.ID, 'q')
    inputElem.clear()

sleep(3)
driver.find_element_by_link_text('로그아웃').click()
sleep(1)
driver.quit()

Found kiscode for 미래에셋큐리어스구조혁신기업재무안정사모투자: 2102604684
No company found for 미래에셋파트너스제구호사모투자. Continuing...
Found kiscode for 동부철구: 065122
No company found for 유진에버베스트턴어라운드기업재무안정사모투자. Continuing...
Found kiscode for 지구레미콘: 783743
Found kiscode for 신세계동대구복합환승센터: 19005993
Found kiscode for 신세계야구단: 136740
Found kiscode for 광주일보사: 1801704720
Found kiscode for 탈리다쿰: 2102609675
No company found for 부신신항다목적터미널. Continuing...
Found kiscode for 두산경영연구원: 1802505387
Found kiscode for 삼성경제연구소: 814130
Found kiscode for 수원삼성축구단: 13072637
Found kiscode for 대구남부에이엠씨: 13077629
No company found for 천안풍세일반산업단지. Continuing...
No company found for 에이치지이니셔티브시몬느임팩트제1호창업베처전문사모투자. Continuing...
Found kiscode for 아이디퀀티크유: 2101603024
Found kiscode for 에스케이에코플랜트: 500283
Found kiscode for 한국넥슬렌유: 1801306247
Found kiscode for 트리엔파트너스: 15091396
Found kiscode for 중흥카이트제십구호위탁관리부동산투자회사: 2303805058
Found kiscode for 롯데미래전략연구소: 1903605826
Found kiscode for 롯데쇼핑타운대구: 1801702681
Found kiscode for 마곡지구피에프브이: 23099248
Found kis

In [16]:
dfOwnInputMrgFirmId['firmnameConv'] = dfOwnInputMrgFirmId['firmnameConv'].replace("에이케이에셋$", '', regex=True)

In [17]:
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '애경중부컨트리클럽'].index, 'firmid'] = 13079298

In [18]:
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '미래에셋파트너스제구호사모투자'].index, 'firmid'] = 1802809331

In [19]:
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '유진에버베스트턴어라운드기업재무안정사모투자'].index, 'firmid'] = 1801904792
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '부신신항다목적터미널'].index, 'firmid'] = 164212
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '천안풍세일반산업단지'].index, 'firmid'] = 2103405848
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '에이치지이니셔티브시몬느임팩트제1호창업베처전문사모투자'].index, 'firmid'] = 2102609014
dfOwnInputMrgFirmId.loc[dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmnameConv'] == '에스에이엠씨투자대부'].index, 'firmid'] = 199741

In [20]:
list(dfOwnInputMrgFirmId[dfOwnInputMrgFirmId['firmid'].isna()]['firmnameConv'].unique())

[]

In [21]:
def convertOwnerNm(row):
    
    out = row['ownername']
    if not isinstance(out, str):
        out = ''
    
    if row['grpname2'] == '엘에스':
        out = re.sub("LS", "엘에스", out)
        out = re.sub("E1", "이원", out)
        out = re.sub("KJ", "케이제이", out)
        out = re.sub("JS", "제이에스", out)
        
    elif row['grpname2'] == '미래에셋':
        out = re.sub('미래에셋파트너스제9호사모투자', '미래에셋파트너스제구호사모투자', out)
        out = re.sub('미래에셋파트너스9호', '미래에셋파트너스제구호사모투자', out)
        
    elif row['grpname2'] == '태영':
        out = re.sub('SBS', '에스비에스', out)
        out = re.sub('KCP', '케이씨피', out)
        out = re.sub('PFV', '피에프브이', out)
        
    elif row['grpname2'] == 'DB':
        out = re.sub('Inc\.*$', '아이엔씨', out)
        out = re.sub('DB', '디비', out)
        
    elif row['grpname2'] == 'KG':
        out = re.sub('KG', '케이지', out)
        
    elif row['grpname2'] == '하림':
        out = re.sub('FS', '에프에스', out)
    
    elif row['grpname2'] == '지에스':
        out = re.sub('GS', '지에스', out)
    
    return out

In [22]:
dfOwnInputMrgFirmId['ownernameConv'] = dfOwnInputMrgFirmId.apply(convertOwnerNm, axis=1)
dfOwnInputMrgFirmId['ownernameConv'] = dfOwnInputMrgFirmId['ownernameConv'].replace("^자중흥주택$", '중흥주택', regex=True)
dfOwnInputMrgFirmId['ownernameConv'] = dfOwnInputMrgFirmId['ownernameConv'].replace("^JTBC", '제이티비씨', regex=True)

In [23]:
dfCmpnySumry['ownernameConv'] = cleanCmpnyNm(dfCmpnySumry['소속회사명']).replace('\s', '', regex=True)

In [24]:
dfOwnInputMrgOwnerId = dfOwnInputMrgFirmId.merge(dfCmpnySumry[['ownernameConv', 'ownerid']].drop_duplicates(), on='ownernameConv', how='left')
dfOwnInputMrgOwnerId.head()

,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode,firmnameConv,firmid,ownernameConv,ownerid
0,그로쓰파트너,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,금융회사,미래에셋,miraeasset,2602105,그로쓰파트너,1701406356.0,미래에셋증권코리아제일호사모투자,1.802203e+09
1,멀티에셋자산운용,0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,,NaN
2,멀티에셋자산운용,1,1.000000,1.000000,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,미래에셋자산운용,4.900500e+05
3,멀티에셋자산운용,99,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,,NaN
4,미래에셋글로벌유니콘사모투자,1,0.867857,0.867857,미래에셋증권,금융회사,미래에셋,miraeasset,2602105,미래에셋글로벌유니콘사모투자,2101109548.0,미래에셋증권,8.600850e+05


In [25]:
list(dfOwnInputMrgOwnerId[(~dfOwnInputMrgOwnerId['ownername'].isna()) & (dfOwnInputMrgOwnerId['ownerid'].isna())]\
     ['ownernameConv'].unique())

['미래에셋파트너스제구호사모투자',
 '동부철구',
 'ICAGroupLimited',
 'H.C.CHOInvestmentLtd.',
 '유진에버베스트턴어라운드기업재무안정사모투자',
 'HanwhaQCELLSCo.,Ltd.',
 '한화생명',
 '트리니티에셋매지니먼트',
 'NCKLOGISTICSHOLDINGPTE.LTD',
 'SPLOGISTICSINVESTMENTPTE.LTD',
 'StarKistCo.',
 'OCI',
 'SGC이테크건설',
 'DoosanBobcatSingaporePte.Ltd.',
 'DoosanHFControlsCorporation',
 '삼성경제연구소',
 'HarmanInternationalIndustries,Inc',
 'NEXONCo.,Ltd.',
 'STOKKEAS',
 '두원홀딩스',
 '한국아트라스비엑스',
 '한국테크놀로지그룹',
 'MitsuiChemicals&SKCPolyurethanesInc.',
 'SK텔레콤',
 'SK에너지',
 'SK가스',
 '에스케이에코플랜트',
 '아이디퀀티크SA',
 'SK',
 'SK머티리얼즈',
 'SKPharmtecoInc.',
 'SABICSKNexleneCompanyPte.Ltd.',
 '롯데홀딩스',
 'L제2투자회사',
 'L제3투자회사',
 '광윤사',
 'L제12투자회사',
 '롯데파이낸셜',
 'L제6투자회사',
 'L제7투자회사',
 'L제9투자회사',
 'L제10투자회사',
 'L제1투자회사',
 'L제4투자회사',
 'L제5투자회사',
 'L제8투자회사',
 'L제11투자회사',
 '패밀리',
 'CatleyInvestmentLtd',
 'HighLoftInternationalLtd',
 'MetrowideInvestmentsLtd',
 'LG전자',
 'LG생활건강',
 'ZKWGroupGmbH',
 'LG',
 'PantosLogisticsChinaCo.,Ltd.',
 'PantosLogisticsHKCompanylimited',
 'PantosLogist

In [26]:
for cmpny in list(dfOwnInputMrgOwnerId[(~dfOwnInputMrgOwnerId['ownername'].isna()) & (dfOwnInputMrgOwnerId['ownerid'].isna())]\
     ['ownernameConv'].unique()):
    
    if re.search('[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+', cmpny):
        print(cmpny)

미래에셋파트너스제구호사모투자
동부철구
유진에버베스트턴어라운드기업재무안정사모투자
한화생명
트리니티에셋매지니먼트
SGC이테크건설
삼성경제연구소
두원홀딩스
한국아트라스비엑스
한국테크놀로지그룹
SK텔레콤
SK에너지
SK가스
에스케이에코플랜트
아이디퀀티크SA
SK머티리얼즈
롯데홀딩스
L제2투자회사
L제3투자회사
광윤사
L제12투자회사
롯데파이낸셜
L제6투자회사
L제7투자회사
L제9투자회사
L제10투자회사
L제1투자회사
L제4투자회사
L제5투자회사
L제8투자회사
L제11투자회사
패밀리
LG전자
LG생활건강
선신
아센디오
이리츠코크렙기업구조조정부동산투자회사
동국
에스엠인터스트리
김영웅
중앙일보엠앰피
한국도서보급
JinroInc.해외
디엘이엔씨
기타동일인관련자
애경피앤티


In [27]:
dfOwnInputMrgOwnerId.shape

(6796, 13)

In [28]:
dfOwnInputMrgOwnerId = dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownername'] != '기타동일인관련자']

In [29]:
dfOwnInputMrgOwnerId.shape

(6795, 13)

In [30]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

grsrch_url = 'https://www.kisline.com/cm/CM0100M00GE00.nice'
driver.get(grsrch_url)
driver.find_element(By.CLASS_NAME, 'btn_close_layer').click()
driver.find_element(By.ID, 'lgnuid').send_keys(username)
driver.find_element(By.ID, 'tmp_lgnupassword').click()
driver.find_element(By.ID, 'lgnupassword').send_keys(password)
driver.find_element(By.CLASS_NAME, 'btn_log_in').click()
driver.implicitly_wait(3)
sleep(1)

inputElem = driver.find_element_by_id('q')

for cmpny in list(dfOwnInputMrgOwnerId[(~dfOwnInputMrgOwnerId['ownername'].isna()) & (dfOwnInputMrgOwnerId['ownerid'].isna())]\
     ['ownernameConv'].unique()):
    if not re.search('[\u3131-\u314e|\u314f-\u3163|\uac00-\ud7a3]+', cmpny) or cmpny.endswith("SA") or cmpny.endswith('해외'):
        dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == cmpny].index, 'ownerid'] = 5350000000
        continue
        
    elif cmpny.startswith("L제"):
        dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == cmpny].index, 'ownerid'] = 2830000000
        continue
        
    inputElem.send_keys(cmpny)
    driver.find_element(By.ID, 'searchView').click()
    driver.implicitly_wait(3)
    try:
        targetRow = driver.find_element(By.XPATH, '//*[@id="eprTable"]/tbody/tr/td[2]')
        kiscode = targetRow.get_attribute('data-kiscode')
        for i in range(len(dfCharToInt)):
            kiscode = re.sub(fr'{dfCharToInt.iloc[i, 0]}', str(dfCharToInt.iloc[i, 1]), kiscode)
        dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == cmpny].index, 'ownerid'] = kiscode
        print(f"Found kiscode for {cmpny}: {kiscode}")
    except:
        print(f"No company found for {cmpny}. Continuing...")
        
    sleep(3)
    inputElem = driver.find_element(By.ID, 'q')
    inputElem.clear()

sleep(3)
driver.find_element_by_link_text('로그아웃').click()
sleep(1)
driver.quit()

No company found for 미래에셋파트너스제구호사모투자. Continuing...
Found kiscode for 동부철구: 065122
No company found for 유진에버베스트턴어라운드기업재무안정사모투자. Continuing...
Found kiscode for 한화생명: 1801604977
No company found for 트리니티에셋매지니먼트. Continuing...
Found kiscode for SGC이테크건설: 501913
Found kiscode for 삼성경제연구소: 814130
Found kiscode for 두원홀딩스: 23075028
Found kiscode for 한국아트라스비엑스: 351296
No company found for 한국테크놀로지그룹. Continuing...
Found kiscode for SK텔레콤: 810185
Found kiscode for SK에너지: 490896
Found kiscode for SK가스: 400122
Found kiscode for 에스케이에코플랜트: 500283
No company found for SK머티리얼즈. Continuing...
Found kiscode for 롯데홀딩스: 1903409008
No company found for 광윤사. Continuing...
No company found for 롯데파이낸셜. Continuing...
Found kiscode for 패밀리: 2101908687
Found kiscode for LG전자: 196275
Found kiscode for LG생활건강: 162210
Found kiscode for 선신: 342745
Found kiscode for 아센디오: 383279
Found kiscode for 이리츠코크렙기업구조조정부동산투자회사: 467595
Found kiscode for 동국: 15067248
No company found for 에스엠인터스트리. Continuing...
No company found

In [31]:
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '유진에버베스트턴어라운드기업재무안정사모투자'].index, 'ownerid'] = 1801904792
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '미래에셋파트너스제구호사모투자'].index, 'ownerid'] = 1802809331
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == 'GS글로벌'].index, 'ownerid'] = 610283
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == 'GS이앤알'].index, 'ownerid'] = 194721
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '트리니티에셋매지니먼트'].index, 'ownerid'] = 901091
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '한국테크놀로지그룹'].index, 'ownerid'] = 351067
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == 'SK머티리얼즈'].index, 'ownerid'] = 93025
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '광윤사'].index, 'ownerid'] = 2830000000
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '롯데파이낸셜'].index, 'ownerid'] = 2830000000
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '김영웅'].index, 'ownerid'] = 2830000000
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '에스엠인터스트리'].index, 'ownerid'] = 768600
dfOwnInputMrgOwnerId.loc[dfOwnInputMrgOwnerId[dfOwnInputMrgOwnerId['ownernameConv'] == '중앙일보엠앰피'].index, 'ownerid'] = 140685

In [32]:
list(dfOwnInputMrgOwnerId[(~dfOwnInputMrgOwnerId['ownername'].isna()) & (dfOwnInputMrgOwnerId['ownerid'].isna())]\
     ['ownernameConv'].unique())

[]

In [33]:
len(dfOwnInputMrgOwnerId)

6795

In [34]:
dfOwnInputMrgOwnerId.head()

,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode,firmnameConv,firmid,ownernameConv,ownerid
0,그로쓰파트너,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,금융회사,미래에셋,miraeasset,2602105,그로쓰파트너,1701406356.0,미래에셋증권코리아제일호사모투자,1802203162.0
1,멀티에셋자산운용,0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,,NaN
2,멀티에셋자산운용,1,1.000000,1.000000,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,미래에셋자산운용,490050.0
3,멀티에셋자산운용,99,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,,NaN
4,미래에셋글로벌유니콘사모투자,1,0.867857,0.867857,미래에셋증권,금융회사,미래에셋,miraeasset,2602105,미래에셋글로벌유니콘사모투자,2101109548.0,미래에셋증권,860085.0


In [35]:
dfOwnInputFin = dfOwnInputMrgOwnerId.drop(columns=['firmnameConv', 'ownernameConv'])
dfOwnInputFin.head()

,firmname,type,own,own2,ownername,금융비금융,grpname2,grpname,grpcode,firmid,ownerid
0,그로쓰파트너,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,금융회사,미래에셋,miraeasset,2602105,1701406356.0,1802203162.0
1,멀티에셋자산운용,0,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,105600.0,NaN
2,멀티에셋자산운용,1,1.000000,1.000000,미래에셋자산운용,금융회사,미래에셋,miraeasset,2602105,105600.0,490050.0
3,멀티에셋자산운용,99,0.000000,0.000000,NaN,금융회사,미래에셋,miraeasset,2602105,105600.0,NaN
4,미래에셋글로벌유니콘사모투자,1,0.867857,0.867857,미래에셋증권,금융회사,미래에셋,miraeasset,2602105,2101109548.0,860085.0


In [36]:
dfGroupNmId21 = pd.read_excel('../data/grpSumry/groupNmId/groupNmId2021.xlsx')
dfGroupNmId21.head()

,grpname2,grpname,grpcode
0,삼성,samsung,511
1,현대자동차,hymotor,1201801
2,에스케이,sk,535
3,엘지,lg,282
4,롯데,lotte,283


In [37]:
len(dfGroupNmId21)

60

In [38]:
dfOwnInputFin.columns

Index(['firmname', 'type', 'own', 'own2', 'ownername', '금융비금융', 'grpname2',
       'grpname', 'grpcode', 'firmid', 'ownerid'],
      dtype='object')

In [39]:
dfOwnInputFin = dfOwnInputFin[['금융비금융', 'grpname2', 'grpname', 'grpcode', 'firmname', 'firmid', 'type',\
                              'own', 'own2', 'ownername', 'ownerid']]

In [40]:
dfOwnInputFin.head()

,금융비금융,grpname2,grpname,grpcode,firmname,firmid,type,own,own2,ownername,ownerid
0,금융회사,미래에셋,miraeasset,2602105,그로쓰파트너,1701406356.0,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,1802203162.0
1,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,0,0.000000,0.000000,NaN,NaN
2,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,1,1.000000,1.000000,미래에셋자산운용,490050.0
3,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,105600.0,99,0.000000,0.000000,NaN,NaN
4,금융회사,미래에셋,miraeasset,2602105,미래에셋글로벌유니콘사모투자,2101109548.0,1,0.867857,0.867857,미래에셋증권,860085.0


In [41]:
dfOwnInputFin.ownerid = dfOwnInputFin.ownerid.astype(float)

In [42]:
dfOwnInputFin.firmid = dfOwnInputFin.firmid.astype(float)

In [43]:
dfOwnInputFin[dfOwnInputFin['own'].isna()]

,금융비금융,grpname2,grpname,grpcode,firmname,firmid,type,own,own2,ownername,ownerid
30,금융회사,미래에셋,miraeasset,2602105,미래에셋증권글로벌제이호사모투자,2.301607e+09,1,NaN,NaN,미래에셋증권,860085.0
73,금융회사,미래에셋,miraeasset,2602105,신성장동력그린퓨처사모투자,1.600360e+07,1,NaN,NaN,미래에셋증권,860085.0
787,비금융회사,지에스,gs,2703408,인아츠프로덕션,2.102304e+09,0,NaN,0.111111,NaN,NaN
811,비금융회사,지에스,gs,2703408,지에스글로벌,6.102830e+05,0,NaN,0.002622,NaN,NaN
821,비금융회사,지에스,gs,2703408,지에스리테일,8.100450e+05,0,NaN,0.000260,NaN,NaN
843,비금융회사,지에스,gs,2703408,지에스홈쇼핑,2.370700e+04,0,NaN,0.091543,NaN,NaN
853,비금융회사,지에스,gs,2703408,텐바이텐,2.085640e+05,0,NaN,0.050015,NaN,NaN
2597,비금융회사,태영,taeyoung,844,디엠씨미디어,2.344750e+05,0,NaN,0.373626,NaN,NaN
2600,비금융회사,태영,taeyoung,844,리앤에스스포츠,1.601548e+07,0,NaN,0.300000,NaN,NaN
2602,비금융회사,태영,taeyoung,844,무노스,2.301805e+09,0,NaN,0.000026,NaN,NaN


In [44]:
dfOwnInputFin.drop([30, 73, 2662, 6160, 6161, 6162])

,금융비금융,grpname2,grpname,grpcode,firmname,firmid,type,own,own2,ownername,ownerid
0,금융회사,미래에셋,miraeasset,2602105,그로쓰파트너,1.701406e+09,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,1.802203e+09
1,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05,0,0.000000,0.000000,NaN,NaN
2,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05,1,1.000000,1.000000,미래에셋자산운용,4.900500e+05
3,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05,99,0.000000,0.000000,NaN,NaN
4,금융회사,미래에셋,miraeasset,2602105,미래에셋글로벌유니콘사모투자,2.101110e+09,1,0.867857,0.867857,미래에셋증권,8.600850e+05
...,...,...,...,...,...,...,...,...,...,...,...
6791,비금융회사,애경,AK,615,코스파,7.320100e+05,1,0.100000,0.100000,에이케이아이에스,3.507960e+05
6792,비금융회사,애경,AK,615,코스파,7.320100e+05,99,0.500000,0.500000,NaN,NaN
6793,비금융회사,애경,AK,615,퍼시픽제3호전문사모부동산투자,1.901203e+09,0,0.000000,0.000000,NaN,NaN
6794,비금융회사,애경,AK,615,퍼시픽제3호전문사모부동산투자,1.901203e+09,1,0.998586,0.998586,제주항공,3.685750e+05


In [45]:
dfOwnInputFin.loc[dfOwnInputFin[dfOwnInputFin['own2'].isna()].index, 'own2'] = 0

In [46]:
dfOwnInputFin.loc[dfOwnInputFin[dfOwnInputFin['own'].isna()].index, 'own'] = 0

In [49]:
grps = ''
for i in range(len(dfGroupNmId21)):
    
    grpname2 = dfGroupNmId21.loc[i, 'grpname2']
    print(grpname2)
    grps = grps + ' ' + grpname2
    df = dfOwnInputFin[dfOwnInputFin['grpname2'] == grpname2]
    
    if not os.path.exists(f'../data/ownership-input-id/2021in/{grpname2}'):
        os.makedirs(f'../data/ownership-input-id/2021in/{grpname2}')
    shutil.copyfile('../data/ownership-input-id/gcentrality.do', f'../data/ownership-input-id/2021in/{grpname2}/gcentrality.do')
    shutil.copyfile('../data/ownership-input-id/gcflowsap.do', f'../data/ownership-input-id/2021in/{grpname2}/gcflowsap.do')
    shutil.copyfile('../data/ownership-input-id/gmatrix.do', f'../data/ownership-input-id/2021in/{grpname2}/gmatrix.do')
    shutil.copyfile('../data/ownership-input-id/gvotes.do', f'../data/ownership-input-id/2021in/{grpname2}/gvotes.do')

    df.to_excel(f'../data/ownership-input-id/2021in/{grpname2}/{grpname2}2021raw.xlsx', index=False)

삼성
현대자동차
에스케이
엘지
롯데
한화
지에스
현대중공업
신세계
씨제이
한진
두산
엘에스
부영
카카오
DL
미래에셋
현대백화점
금호아시아나
셀트리온
한국투자금융
교보생명보험
네이버
에이치디씨
효성
영풍
하림
케이씨씨
넥슨
넷마블
호반건설
SM
DB
코오롱
한국타이어
오씨아이
태영
이랜드
세아
중흥건설
태광
동원
한라
아모레퍼시픽
IMM인베스트먼트
삼천리
금호석유화학
다우키움
장금상선
동국제강
애경
반도홀딩스
유진
하이트진로
삼양
대방건설
현대해상화재보험
엠디엠
아이에스지주
중앙


In [50]:
grps = grps[1:]
grps

'삼성 현대자동차 에스케이 엘지 롯데 한화 지에스 현대중공업 신세계 씨제이 한진 두산 엘에스 부영 카카오 DL 미래에셋 현대백화점 금호아시아나 셀트리온 한국투자금융 교보생명보험 네이버 에이치디씨 효성 영풍 하림 케이씨씨 넥슨 넷마블 호반건설 SM DB 코오롱 한국타이어 오씨아이 태영 이랜드 세아 중흥건설 태광 동원 한라 아모레퍼시픽 IMM인베스트먼트 삼천리 금호석유화학 다우키움 장금상선 동국제강 애경 반도홀딩스 유진 하이트진로 삼양 대방건설 현대해상화재보험 엠디엠 아이에스지주 중앙'

In [48]:
for i in range(len(dfGroupNmId21)):
    
    grpname2 = dfGroupNmId21.loc[i, 'grpname2']
    df = dfOwnInputFin[dfOwnInputFin['grpname2'] == grpname2]
    
    if not os.path.exists(f'../data/almeida/2021/in'):
        os.makedirs(f'../data/almeida/2021/in')
    
    if i < 9:
        iStr = '0'+str(i+1)
    else:
        iStr = str(i+1)
    fileNm = f'../data/almeida/2021/in/{iStr}.{grpname2}2021.xlsx'
    print(fileNm)
    df.to_excel(fileNm, index=False)

../data/almeida/2021/in/01.삼성2021.xlsx
../data/almeida/2021/in/02.현대자동차2021.xlsx
../data/almeida/2021/in/03.에스케이2021.xlsx
../data/almeida/2021/in/04.엘지2021.xlsx
../data/almeida/2021/in/05.롯데2021.xlsx
../data/almeida/2021/in/06.한화2021.xlsx
../data/almeida/2021/in/07.지에스2021.xlsx
../data/almeida/2021/in/08.현대중공업2021.xlsx
../data/almeida/2021/in/09.신세계2021.xlsx
../data/almeida/2021/in/10.씨제이2021.xlsx
../data/almeida/2021/in/11.한진2021.xlsx
../data/almeida/2021/in/12.두산2021.xlsx
../data/almeida/2021/in/13.엘에스2021.xlsx
../data/almeida/2021/in/14.부영2021.xlsx
../data/almeida/2021/in/15.카카오2021.xlsx
../data/almeida/2021/in/16.DL2021.xlsx
../data/almeida/2021/in/17.미래에셋2021.xlsx
../data/almeida/2021/in/18.현대백화점2021.xlsx
../data/almeida/2021/in/19.금호아시아나2021.xlsx
../data/almeida/2021/in/20.셀트리온2021.xlsx
../data/almeida/2021/in/21.한국투자금융2021.xlsx
../data/almeida/2021/in/22.교보생명보험2021.xlsx
../data/almeida/2021/in/23.네이버2021.xlsx
../data/almeida/2021/in/24.에이치디씨2021.xlsx
../data/almeida/2021/in/25.효

In [49]:
df = pd.read_excel("../data/almeida/2021/out/stacked2021.xlsx")
df.head()

,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central,grpname2
0,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265,유진
1,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102,유진
2,165401,3,0.388302,2.172676,2.0,NaN,0.0,38,1.551020,유진
3,166364,4,1.000000,1.000000,1.0,NaN,0.0,100,1.632652,유진
4,202487,5,0.125871,3.012328,2.0,NaN,0.0,29,0.000000,유진


In [50]:
dfOwnInputFin.head()

,금융비금융,grpname2,grpname,grpcode,firmname,firmid,type,own,own2,ownername,ownerid
0,금융회사,미래에셋,miraeasset,2602105,그로쓰파트너,1.701406e+09,1,1.000000,1.000000,미래에셋증권코리아제일호사모투자,1.802203e+09
1,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05,0,0.000000,0.000000,NaN,NaN
2,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05,1,1.000000,1.000000,미래에셋자산운용,4.900500e+05
3,금융회사,미래에셋,miraeasset,2602105,멀티에셋자산운용,1.056000e+05,99,0.000000,0.000000,NaN,NaN
4,금융회사,미래에셋,miraeasset,2602105,미래에셋글로벌유니콘사모투자,2.101110e+09,1,0.867857,0.867857,미래에셋증권,8.600850e+05


In [51]:
df['year'] = 2020
df['year2'] = 2021

In [52]:
dfGrp = pd.read_excel('../data/grpSumry/groupNmId/groupNmId2021.xlsx')
dfGrp.head()

,grpname2,grpname,grpcode
0,삼성,samsung,511
1,현대자동차,hymotor,1201801
2,에스케이,sk,535
3,엘지,lg,282
4,롯데,lotte,283


In [53]:
df = df.merge(dfGrp, on='grpname2', how='left')
df.head()

,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central,grpname2,year,year2,grpname,grpcode
0,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265,유진,2020,2021,eugenes,653
1,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102,유진,2020,2021,eugenes,653
2,165401,3,0.388302,2.172676,2.0,NaN,0.0,38,1.551020,유진,2020,2021,eugenes,653
3,166364,4,1.000000,1.000000,1.0,NaN,0.0,100,1.632652,유진,2020,2021,eugenes,653
4,202487,5,0.125871,3.012328,2.0,NaN,0.0,29,0.000000,유진,2020,2021,eugenes,653


In [54]:
df = df.merge(dfOwnInputFin[['firmid', 'firmname']], on='firmid', how='left')
df.head()

,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central,grpname2,year,year2,grpname,grpcode,firmname
0,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265,유진,2020,2021,eugenes,653,케이비에프
1,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265,유진,2020,2021,eugenes,653,케이비에프
2,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265,유진,2020,2021,eugenes,653,케이비에프
3,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102,유진,2020,2021,eugenes,653,한성레미콘
4,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102,유진,2020,2021,eugenes,653,한성레미콘


In [56]:
df = df[['year', 'year2', 'grpname2', 'grpname', 'grpcode', 'firmname', 'firmid', 'fid', 'ultown', 'avpos', 'sd', 'steps', 'loop', 'lastcontrol', 'central']]
df.head()

,year,year2,grpname2,grpname,grpcode,firmname,firmid,fid,ultown,avpos,sd,steps,loop,lastcontrol,central
0,2020,2021,유진,eugenes,653,케이비에프,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265
1,2020,2021,유진,eugenes,653,케이비에프,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265
2,2020,2021,유진,eugenes,653,케이비에프,5086,1,0.003715,4.142763,3.0,NaN,0.0,8,0.163265
3,2020,2021,유진,eugenes,653,한성레미콘,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102
4,2020,2021,유진,eugenes,653,한성레미콘,102640,2,0.103584,3.275612,3.0,NaN,0.0,29,0.755102


In [60]:
df.to_excel('../data/almeida/2021/out/almeida_own1_2021.xlsx', index=False)